In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split


In [2]:
df_train_full=pd.read_csv('../input/train.csv')
print(len(df_train_full))

df_train_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
print(len(df_train_full))

df_test_full=pd.read_csv('../input/test.csv')

1460
1460


In [3]:
y = df_train_full.SalePrice
df_train_full.drop(['SalePrice'], axis=1, inplace=True)
X_train, X_valid, y_train, y_valid = train_test_split(df_train_full, y, train_size=0.8, test_size=0.2,random_state=0)
X_test=df_test_full

In [4]:
print(len(X_train.index))
print(len(df_train_full.columns))
print(len(y_train))


1168
80
1168




**Taking care of null values**

  Those columns that have more than 75% missing values will be dropped


In [7]:
from utils import correct_nan
missing_val_count_by_column = (X_train.isnull().sum(axis=0))
# print(missing_val_count_by_column['Fence'])

cols_with_missing_values=np.array(missing_val_count_by_column[missing_val_count_by_column > 0].index)
print('Columns with missing values: {}'.format((cols_with_missing_values)))
 
cols_with_missing_more_than_75_percent =np.array(missing_val_count_by_column[missing_val_count_by_column > len(X_train.index)*(3/4)].index)
print('Columns with more than 75 percent missing values: {}'.format((cols_with_missing_more_than_75_percent)))

remaining_missing_cols=np.setdiff1d(cols_with_missing_values,cols_with_missing_more_than_75_percent)
print(remaining_missing_cols)

X_train=correct_nan(X_train)
X_valid=correct_nan(X_valid)
X_test=correct_nan(X_test)




Columns with missing values: ['LotFrontage' 'MasVnrType' 'MasVnrArea' 'Electrical' 'GarageYrBlt'
 'GarageFinish']
Columns with more than 75 percent missing values: []
['Electrical' 'GarageFinish' 'GarageYrBlt' 'LotFrontage' 'MasVnrArea'
 'MasVnrType']


In [9]:
special_columns=np.array(['Alley','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType',
                 'GarageFinish'  ,'GarageQual','GarageCond','PoolQC','Fence','MiscFeature'])
for col in special_columns:
    dic={}
    for ind,val in X_train[col].iteritems():
        dic[val] = dic.get(val, 0) + 1
    print(col,dic)

Alley {'NoAlley': 1097, 'Grvl': 37, 'Pave': 34}
BsmtQual {'Ex': 94, 'TA': 528, 'Gd': 490, 'NoBasement': 28, 'Fa': 28}
BsmtCond {'TA': 1046, 'Gd': 55, 'NoBasement': 28, 'Fa': 37, 'Po': 2}
BsmtExposure {'Av': 174, 'No': 768, 'Gd': 106, 'NoBasement': 28, 'Mn': 92}
BsmtFinType1 {'GLQ': 330, 'Unf': 353, 'ALQ': 172, 'Rec': 106, 'NoBasement': 28, 'LwQ': 56, 'BLQ': 123}
BsmtFinType2 {'Unf': 1003, 'ALQ': 14, 'Rec': 39, 'NoBasement': 29, 'BLQ': 30, 'LwQ': 42, 'GLQ': 11}
FireplaceQu {'Gd': 295, 'NoFireplace': 551, 'TA': 257, 'Ex': 19, 'Po': 17, 'Fa': 29}
GarageType {'Attchd': 696, 'Detchd': 315, 'BuiltIn': 74, 'NoGarage': 58, 'Basment': 14, '2Types': 6, 'CarPort': 5}
GarageFinish {'Unf': 488, 'RFn': 337, 'Fin': 285, nan: 58}
GarageQual {'TA': 1055, 'Fa': 40, 'NoGarage': 58, 'Gd': 10, 'Ex': 2, 'Po': 3}
GarageCond {'TA': 1063, 'Fa': 32, 'NoGarage': 58, 'Gd': 8, 'Ex': 1, 'Po': 6}
PoolQC {'NoPool': 1164, 'Fa': 1, 'Gd': 2, 'Ex': 1}
Fence {'NoFence': 954, 'GdWo': 43, 'GdPrv': 51, 'MnPrv': 113, 'MnWw': 

In [10]:

categorical_cols = [cname for cname in X_train.columns if X_train[cname].dtype == "object"]
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]
print(len(categorical_cols))
print(len(numerical_cols))

43
37


In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [12]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', xgb_model)
                             ])

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='constant',
                                                                verbose=0),
                                                  ['Id', 'MSSubClass',
                                                   'LotFrontage', 'LotArea',
                                                   'OverallQual', 'Ove

In [13]:
from sklearn.metrics import mean_absolute_error
preds = pipeline.predict(X_valid)
# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 16252.772153253425


In [14]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

In [15]:
mae_compare = pd.Series()
mae_compare.index.name = 'Algorithm'

mae_compare['XGBRegressor'] = score


rf_model = RandomForestRegressor(random_state=3)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('rf_model', rf_model)
                          ])
pipeline.fit(X_train, y_train)
rf_val_mae = mean_absolute_error(pipeline.predict(X_valid), y_valid)
mae_compare['RandomForest'] = rf_val_mae




linear_model = LinearRegression()
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('linear_model', linear_model)
                          ])
pipeline.fit(X_train, y_train)
linear_val_mae = mean_absolute_error(pipeline.predict(X_valid), y_valid)
mae_compare['LinearRegression'] = linear_val_mae




lasso_model = Lasso(alpha=0.0005, random_state=5)
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('lasso_model', lasso_model)
                          ])
pipeline.fit(X_train, y_train)
lasso_val_mae = mean_absolute_error(pipeline.predict(X_valid), y_valid)
mae_compare['Lasso'] = lasso_val_mae



print('MAE values for different algorithms:')
mae_compare.sort_values(ascending=True).round()


<ipython-input-15-3a8f319f46f8>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mae_compare = pd.Series()


MAE values for different algorithms:


C:\Users\Dimitar\anaconda3\envs\myenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:457: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 197984029804.82953, tolerance: 719132949.923008
  model = cd_fast.sparse_enet_coordinate_descent(


Algorithm
XGBRegressor        16253.0
RandomForest        17527.0
Lasso               22696.0
LinearRegression    23489.0
dtype: float64

In [16]:

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', xgb_model)
                             ])
pipeline.fit(X_train, y_train)

predicts= pipeline.predict(X_test)
sub=pd.read_csv('../output/xgboost_submission.csv')

print(sub.SalePrice)
print(predicts)

# sub_df=pd.DataFrame({'Id':X_test.Id,'SalePrice':predicts})
# sub_df.to_csv('../output/xgboost_submission.csv',index=False)

0       126967.980
1       153094.530
2       190674.800
3       190710.310
4       195083.470
           ...    
1454     77490.740
1455     82617.086
1456    164062.250
1457    109265.140
1458    222919.020
Name: SalePrice, Length: 1459, dtype: float64
[123942.11 156881.88 194983.81 ... 164560.03 110870.49 226929.89]
